In [46]:
import numpy as np
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [47]:
df = pd.read_csv('Preprocessed Fake Reviews Detection Dataset.csv')
df['text_'].fillna('', inplace=True)
reviews = df['text_'].values
categories = df['category'].values
sentiment =df['sentiment']
label = df['label'].values
df['label'] = df['label'].map({'CG': 0, 'OR': 1})
df['label'] = tf.keras.utils.to_categorical(df['label'])
df

,Unnamed: 0,category,rating,label,text_,sentiment
0,0,Home_and_Kitchen_5,5.0,1.0,love well made sturdi comfort i love veri pretti,0.9592
1,1,Home_and_Kitchen_5,5.0,1.0,love great upgrad origin i 've mine coupl year,0.8910
2,2,Home_and_Kitchen_5,5.0,1.0,thi pillow save back i love look feel pillow,0.7906
3,3,Home_and_Kitchen_5,1.0,1.0,miss inform use great product price i,0.7463
4,4,Home_and_Kitchen_5,5.0,1.0,veri nice set good qualiti we set two month,0.7397
...,...,...,...,...,...,...
40427,40427,Clothing_Shoes_and_Jewelry_5,4.0,0.0,i read review say bra ran small i order two ba...,0.9943
40428,40428,Clothing_Shoes_and_Jewelry_5,5.0,1.0,i n't sure exactli would it littl larg small s...,0.9988
40429,40429,Clothing_Shoes_and_Jewelry_5,2.0,0.0,you wear hood wear hood wear jacket without ho...,0.6892
40430,40430,Clothing_Shoes_and_Jewelry_5,1.0,1.0,i like noth dress the reason i gave star i ord...,0.9988


In [48]:

num_words = 10000  
max_len = 100  

tokenizer = Tokenizer(num_words=num_words, oov_token='<OOV>')
tokenizer.fit_on_texts(reviews)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(reviews)
padded_sequences = pad_sequences(sequences, maxlen=max_len)

df


,Unnamed: 0,category,rating,label,text_,sentiment
0,0,Home_and_Kitchen_5,5.0,1.0,love well made sturdi comfort i love veri pretti,0.9592
1,1,Home_and_Kitchen_5,5.0,1.0,love great upgrad origin i 've mine coupl year,0.8910
2,2,Home_and_Kitchen_5,5.0,1.0,thi pillow save back i love look feel pillow,0.7906
3,3,Home_and_Kitchen_5,1.0,1.0,miss inform use great product price i,0.7463
4,4,Home_and_Kitchen_5,5.0,1.0,veri nice set good qualiti we set two month,0.7397
...,...,...,...,...,...,...
40427,40427,Clothing_Shoes_and_Jewelry_5,4.0,0.0,i read review say bra ran small i order two ba...,0.9943
40428,40428,Clothing_Shoes_and_Jewelry_5,5.0,1.0,i n't sure exactli would it littl larg small s...,0.9988
40429,40429,Clothing_Shoes_and_Jewelry_5,2.0,0.0,you wear hood wear hood wear jacket without ho...,0.6892
40430,40430,Clothing_Shoes_and_Jewelry_5,1.0,1.0,i like noth dress the reason i gave star i ord...,0.9988


In [51]:

x_trains, x_tests, y_train, y_test = train_test_split(padded_sequences, label, test_size=0.2, random_state=42)

x_trains_list = x_trains.tolist()
x_tests_list = x_tests.tolist()

# Create an instance of TfidfVectorizer
vectorizer = TfidfVectorizer()

# Fit the vectorizer on the text reviews and transform the text reviews into TF-IDF features
x_train = vectorizer.fit_transform(x_trains_list)
x_test = vectorizer.transform(x_tests_list)


embedding_dim = 100  # Dimensionality of the word embeddings
lstm_units = 128  # Number of units in the LSTM layer

model = Sequential()
model.add(Embedding(num_words, embedding_dim, input_length=max_len))
model.add(LSTM(lstm_units))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 3  # Number of training epochs
batch_size = 30  # Batch size for training

model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_test, y_test))

AttributeError: 'list' object has no attribute 'lower'

In [ ]:

loss, accuracy = model.evaluate(x_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')